# Three qubit repetition code
This sample demonstrates how to implement a 3 qubit repetition code that can be used to detect and correct bit flip errors. It leverages integrated hybrid computing features to count the number of times a bit flip occurred while qubits are coherent.

Find more about integrated quantum computing at https://aka.ms/AQ/Hybrid/Docs

In this sample we will:
1. Implement a 3 qubit repetition code.
2. Detect and correct bit flip errors.
3. Use classical compute to count the number of times a bit flip occurred.
## Connect to the Azure Quantum workspace
First, find the resource ID of your Azure Quantum workspace. You can copy/paste this from the top-right corner of your Quantum Workspace page in Azure Portal.

Next, you can run `%azure.connect` to connect to the workspace and display the list of provisioned targets that support execution of Q# programs. If you are prompted to login, be sure to use the same account you used to create your Azure Quantum workspace.

In [ ]:
%azure.connect "/subscriptions/subscriptionId/resourceGroups/resourceGroupName/providers/Microsoft.Quantum/Workspaces/workspaceName" location="westus"

## Select a target and opt-in to use integrated hybrid computing
Now, we select the target the program is intended to be run on using the `%azure.target` command. This information is used by the Q# compiler to check the program can be run on the specified target taking into consideration its capabilities.

Since integrated hybrid computing is an opt-in feature, we run the `%azure.target-capability` command to opt into it.


In [ ]:
%azure.target quantinuum.sim.h1-1e
%azure.target-capability AdaptiveExecution

## Open the required Q# namespaces
Open the Q# namespaces that provide the necessary operations to implement this sample.


In [ ]:
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Measurement;

## Implement a repetition code
The following operation is used to create a repetition code using a 3 qubit register.

In [ ]:
operation Encode(register : Qubit[]) : Unit is Adj {
    CNOT(register[0], register[1]);
    CNOT(register[0], register[2]);
}

## Measure a bit flip error syndrome
The following operation measures a bit flip error syndrome by checking the parities between qubits 0 and 1, and between qubits 1 and 2.

This operation uses a 2 qubit auxiliary register and leverages two integrated hybrid computing capabilities: mid-program qubit measurement and qubit re-use. For some programs, these features reduce the number of qubits needed.

In [ ]:
operation MeasureBitFlipSyndrome(encodedRegister : Qubit[], auxiliaryRegister : Qubit[]) : (Result, Result)
{
    // Measure the bit flip syndrome by checking the parities between qubits 0 and 1, and between qubits 1 and 2.
    ResetAll(auxiliaryRegister);
    CNOT(encodedRegister[0], auxiliaryRegister[0]);
    CNOT(encodedRegister[1], auxiliaryRegister[0]);
    CNOT(encodedRegister[1], auxiliaryRegister[1]);
    CNOT(encodedRegister[2], auxiliaryRegister[1]);
    let parity01 = MResetZ(auxiliaryRegister[0]);
    let parity12 = MResetZ(auxiliaryRegister[1]);
    return (parity01, parity12);
}


## Use the bit flip error syndrome to revert bit flips
The following operation reverts a bit flip in a specific qubit in the register depending on the provided error syndrome. It returns a boolean that represents whether any a bit flip was reverted on any of the qubits
.
This operation uses another integrated hybrid computing capability, branching based on qubit measurements. As we can observe, a different quantum operation is performed depending on the measurement results received as parameters.

In [ ]:
operation RevertBitFlip(register : Qubit[], parity01 : Result, parity12 : Result) : Bool
{
    if (parity01 == One and parity12 == Zero) {
        X(register[0]);
    }
    elif (parity01 == One and parity12 == One) {
        X(register[1]);
    }
    elif (parity01 == Zero and parity12 == One) {
        X(register[2]);
    }

    return parity01 == One or parity12 == One;
}


## Implement an operation that could introduce bit flips
In order to test our repetition code, we can apply an operation to our encoded qubit register that is equivalent to applying an identity operation.
To achieve this, we can apply a few rotations about the x-axis such that after applying them the qubit register is in the same state it was originally in.
Since the `Rx` operation has a $2\pi$ period, applying a $\frac{\pi}{2}$ rotation 4 times should return the qubit to its original state (given that it is not possible to measure the difference between states $|\psi〉$ and $-|\psi〉$).
Note that since the qubit register represents a repetition code, rotations are applied to each qubit in the register.

When running this operation on real quantum devices or noisy simulators, unitended bit flips could occur, and we want our program to detect and correct them.


In [ ]:
operation ApplyRotationalIdentity(register : Qubit[]) : Unit is Adj
{
    let theta = PI() * 0.5;
    for i in 1 .. 4 {
        for qubit in register
        {
            Rx(theta, qubit);
        }
    }
}


## Create a program to test our repetition code
Now that we have implemented all the building blocks, we will integrate them all into a single program that we can submit to a quantum device or simulator.

Our program will do the following:
1. Initialize a qubit into a superposition state $|-〉$.
2. Create 3 qubit repetition code.
3. Apply several quantum operations to the encoded qubit register, performing bit flip detection and correction between each quantum operation.
4. Decode the qubit register and measure it in the computational basis.
5. As output, return a boolean representing whether the qubit measurement was the expected one and an integer representing the number of times a bit flip occurred.

This program uses two more integrated hybrid computing features: classical compute during coherence time and additional output data types (boolean and integer).
### ❕ Please note that you will see a warning displayed (QS5028)
It can be ignored for the purposes of this sample. It will be addressed in a subsequent release of the Quantum Development Kit.

In [ ]:
operation ThreeQubitRepetitionCode() : (Bool, Int) {
    // Use two qubit registers, one for encoding and an auxiliary one for syndrome measurements.
    use encodedRegister = Qubit[3];
    use auxiliaryRegister = Qubit[2];

    // Initialize the first qubit in the register to a |-〉 state.
    H(encodedRegister[0]);
    Z(encodedRegister[0]);

    // Apply several unitary operations to the encoded qubits performing bit flip detection and correction between
    // each application.
    mutable bitFlipCount = 0;
    within {
        // The 3 qubit register is used as a repetition code.
        Encode(encodedRegister);
    }
    apply {
        let iterations = 5;
        for _ in 1 .. iterations {
            // Apply a sequence of rotations to the encoded register that effectively perform an identity operation.
            ApplyRotationalIdentity(encodedRegister);

            // Measure the bit flip error syndrome, revert the bit flip if needed, and increase the count if a bit flip occurred.
            let (parity01, parity12) = MeasureBitFlipSyndrome(encodedRegister, auxiliaryRegister);
            let bitFlipReverted = RevertBitFlip(encodedRegister, parity01, parity12);
            if (bitFlipReverted) {
                set bitFlipCount += 1;
            }
        }
    }

    // Transform the qubit to the |1〉 state and measure it in the computational basis.
    H(encodedRegister[0]);
    let result = MResetZ(encodedRegister[0]) == One;
    ResetAll(encodedRegister);

    // The output of the program is a boolean-integer tuple where the boolean represents whether the qubit
    // measurement result was the expected one and the integer represents the number of times bit flips occurred
    // throughout the program.
    return (result, bitFlipCount);
}


## Submit the three qubit repetition code program
### ❕ This program will use approximately 11.31 units (HQCs or eHQCs) using 50 shots, as configured below.
If the job has not been completed after the client times out (300 seconds), you can query the status using `%azure.status` followed by querying the output using `%azure.output`.<br/>

If the kernel was restarted after the program had been submitted, you will have to use `%azure.status <job_id>` to get the status of the job and `%azure.output <job_id>` to retrieve the results.<br/>
You can also check the status of the job under the **Job management** section on the left side menu of the portal.


In [ ]:
%azure.execute ThreeQubitRepetitionCode shots=50 timeout=300 poll=10

In [ ]:
%azure.status

## Understanding the results
Each item in the histogram is a boolean-integer tuple representing whether the qubit measurement was the expected one and the number of times bit flip occurred throughout program execution. Associated to an item in the histogram is the frequency of that particular output.<br/>

For example:
- A `(true, 0)` output means that the qubit was measured in the expected state and no bit flips occurred.
- A `(true, 2)` output means that the qubit was measured in the expected state and bit flips occurred twice.
- A `(false, 1)` output means that the qubit was **not** measured in the expected state and bit flips occurred once.


In [ ]:
%azure.output

## ↗ Learning more about Integrated Hybrid Computing
Now that you have run this sample, you can further learn and experiment with other samples available in the gallery.

To learn more, please refer to our [online documentation](https://aka.ms/AQ/Hybrid/Docs).